In [ ]:
import math
import os
import torch
import torch.distributions.constraints as constraints
import pyro
from pyro.optim import Adam
from pyro.infer import SVI, Trace_ELBO
import pyro.distributions as dist
import csv


Let's return to and extend [the coin-toss example from the pyro tutorial](http://pyro.ai/examples/svi_part_i.html)

$$\begin{aligned}
\operatorname{fairness} &\sim \operatorname{Beta}(10,10)\\
\operatorname{measurement} &\sim \operatorname{Binom}(\operatorname{fairness}).
\end{aligned}$$

We have in fact tossed the coin 100 times in our special coin tossing laboratory, and recorded the data in a CSV file.

In [ ]:
# load some data from our csv file
tosses = []
with open('coin_tosses.csv', newline='') as f:
    reader = csv.reader(f)
    next(reader) #skip header
    for row in reader:
        tosses.append(int(row[0]))
print(tosses)


In [ ]:

n_steps =2000

assert pyro.__version__.startswith('1.7.0')

# clear the param store in case we're in a REPL
pyro.clear_param_store()



def model(data):
    # define the hyperparameters that control the beta prior
    alpha0 = torch.tensor(10.0)
    beta0 = torch.tensor(10.0)
    # sample f from the beta prior
    f = pyro.sample("latent_fairness", dist.Beta(alpha0, beta0))
    # loop over the observed data
    for i in range(len(data)):
        # observe datapoint i using the bernoulli likelihood
        pyro.sample("obs_{}".format(i), dist.Bernoulli(f), obs=data[i])

def guide(data):
    # register the two variational parameters with Pyro
    # - both parameters will have initial value 15.0.
    # - because we invoke constraints.positive, the optimizer
    # will take gradients on the unconstrained parameters
    # (which are related to the constrained parameters by a log)
    alpha_q = pyro.param("alpha_q", torch.tensor(15.0),
                         constraint=constraints.positive)
    beta_q = pyro.param("beta_q", torch.tensor(15.0),
                        constraint=constraints.positive)
    # sample latent_fairness from the distribution Beta(alpha_q, beta_q)
    pyro.sample("latent_fairness", dist.Beta(alpha_q, beta_q))

# setup the optimizer
adam_params = {"lr": 0.0005, "betas": (0.90, 0.999)}
optimizer = Adam(adam_params)

# setup the inference algorithm
svi = SVI(model, guide, optimizer, loss=Trace_ELBO())

# do gradient steps
for step in range(n_steps):
    svi.step(data)
    if step % 100 == 0:
        print('.', end='')

# grab the learned variational parameters
alpha_q = pyro.param("alpha_q").item()
beta_q = pyro.param("beta_q").item()

# here we use some facts about the beta distribution
# compute the inferred mean of the coin's fairness
inferred_mean = alpha_q / (alpha_q + beta_q)
# compute inferred standard deviation
factor = beta_q / (alpha_q * (1.0 + alpha_q + beta_q))
inferred_std = inferred_mean * math.sqrt(factor)

print("\nbased on the data and our prior belief, the fairness " +
      "of the coin is %.3f +- %.3f" % (inferred_mean, inferred_std))

Here we took an object with a true weight of 8 and got some noisy measurement.
We then evaluated the probability of observing that measurement. We can easily
use this functionality to plot the PDF of our scale and the object.

In [ ]:
lb = weight - noise * 4
ub = weight + noise * 4
resolution = 1000
measurements = torch.arange(lb, ub, (ub - lb) / resolution)
probs = scale_dist.log_prob(measurements).exp()
plt.plot(measurements, probs, color="red")
ones = torch.ones(100000)
multiscale_dist = make_scale_dist(weight * ones, noise * ones)
samples = pyro.sample("sample", multiscale_dist)
plt.hist(samples.cpu().numpy(), bins=100, density=True, color="blue", alpha=0.5)
plt.show()

The PDF looks like a Gaussian centered at $x=8$, and lines up closely with the
sample histogram. Now let's examine a more complicated model, where we have a
good guess of the object's weight. Mathematically, this becomes

$$ \operatorname{weight} | \operatorname{guess} \sim \mathcal{N}(\operatorname{guess}, 1) $$

$$ \operatorname{measurement} | \operatorname{weight} \sim \mathcal{N}(\operatorname{weight}, 0.75) $$


In [ ]:
pyro.clear_param_store()
pyro.set_rng_seed(101)

def model_1(guess):
   weight = pyro.sample("weight", dist.Normal(guess, 1.0))
   y = pyro.sample("measurement", dist.Normal(weight, 0.75))
   return y

The `trace` class lets us maintain a trace of the entire model, so we can query
the particular terms we care about without having to pass them around between
different name spaces.

In [ ]:
guess = torch.tensor(7.5)
trace_1 = poutine.trace(model_1).get_trace(guess)
trace_1.compute_log_prob()
nodes =  trace_1.nodes
print(f"weight: {nodes['weight']['value']},"
      f" probability {nodes['weight']['log_prob'].exp()}")
print(f"measurement: {nodes['measurement']['value']},"
      f" probability {nodes['measurement']['log_prob'].exp()}")

We can also condition the model to enforce that certain RVs will always have a
specific value. This is done with the `condition` API.

In [ ]:
cond_model = pyro.condition(model_1,
                            data={"measurement": torch.tensor(9.5)})
cond_trace = poutine.trace(cond_model).get_trace(guess)
cond_trace.compute_log_prob()
nodes = cond_trace.nodes
print(f"weight: {nodes['weight']['value']},"
      f" likelihood {nodes['weight']['log_prob'].exp()}")
print(f"measurement: {nodes['measurement']['value']},"
      f" likelihood {nodes['measurement']['log_prob'].exp()}")

We conditioned the `measurement` RV to evaluate to `9.5` so it was not sampled
from its respective distribution. Note, however, that this does not affect the
probability of the observation. The probability returned by pyro is
$p(measurement = 9.5 | \mathcal{N}(weight, 0.75)$, and not `1`, even though
there was a `100%` chance of `measurement` evaluating to `9.5`. Similarly, if we
remove the conditioning on `measurement`, we see that the likelihood of `weight`
does not change, even though $p(\operatorname{weight} | \operatorname{measurement}) \neq p(\operatorname{weight})$.

In [ ]:
cond_model = pyro.condition(model_1,
                            data={"weight": nodes["weight"]["value"]})
trace = poutine.trace(cond_model).get_trace(guess)
trace.compute_log_prob()
weight_node = trace.nodes["weight"]
print(f"weight: {weight_node['value']}, likelihood: {weight_node['log_prob'].exp()}")

measurement_node = trace.nodes["measurement"]
print(f"measurement: {measurement_node['value']}, likelihood: {measurement_node['log_prob'].exp()}")


It's important to remember that the `condition` function does not actually
change the underlying distribution of the RV. Think of it as saying "let's
pretend that we happend to sample `X` for this variable". Hence, RVs upstream
of the conditioned variable do not change their likelihood. The same is not true
 of downstream RVs.

In [ ]:
trace = poutine.trace(model_1).get_trace(guess)
measurements = torch.arange(lb, ub, (ub - lb) / resolution)
measurement_dist = trace.nodes['measurement']['fn']
probs = measurement_dist.log_prob(measurements).exp()
plt.plot(measurements, probs, color="blue")

cond_model = pyro.condition(model_1, data={"weight": 9})
cond_trace = poutine.trace(cond_model).get_trace(guess)
cond_dist = cond_trace.nodes['measurement']['fn']
cond_probs = cond_dist.log_prob(measurements).exp()
plt.plot(measurements, cond_probs, color="red")

plt.show()

Here we plotted the PDF of `measurement` in both an unconditional model (blue)
and a model conditioned on `weight = 9` (red). We can see that conditioning on
`weight` changes the probability distribution of the downstream RV
`measurement`. Repeatedly executing the above cell will produce a new
unconditional PDF every time, because it depends on the stochastic sample from
$p(weight | guess=7.5)$. The conditional PDF won't change because it samples
deterministically from $p(weight = 9) = 1$

* http://pyro.ai/examples/bayesian_regression.html
* http://pyro.ai/examples/bayesian_regression_ii.html
* http://pyro.ai/examples/intro_part_i.html
* http://pyro.ai/examples/intro_part_ii.html
* http://pyro.ai/examples/effect_handlers.html
* http://pyro.ai/examples/sir_hmc.html
* https://docs.pyro.ai/en/1.7.0/poutine.html
* http://pyro.ai/examples/mle_map.html